In [1]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-5000.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)



num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)


result_df_op=result_df

X=result_df_op
y=churn

In [3]:
#pip install mlencoders
from mlencoders.weight_of_evidence_encoder import WeightOfEvidenceEncoder

enc = WeightOfEvidenceEncoder(cols=X.columns)
X_encoded = enc.fit_transform(X, y)
X_encoded.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.155052,-0.022609,-1.394963,0.00000,-0.023140,0.00000,0.0,-0.113212,0.0,0.194272,-0.320183,0.011951,-0.365343,-0.088010,-0.365343,-0.324521,0.299633,-0.264597,-0.678742
1,-0.473557,-0.022609,-0.355774,0.00000,0.194272,0.00000,0.0,-0.458053,0.0,0.417416,-0.057042,0.417416,1.110563,-0.088010,1.110563,-0.324521,-0.103360,-0.264597,-0.678742
2,0.011951,-0.022609,0.178529,0.00000,-0.055188,0.00000,0.0,0.117311,0.0,0.194272,0.000901,0.194272,-0.458053,-0.225721,-0.458053,-0.205114,0.705098,-0.264597,0.178855
3,-0.365343,-0.006706,0.178529,0.00000,0.045852,0.00000,0.0,0.242475,0.0,0.000000,0.078200,0.000000,-0.937130,-0.069966,-0.937130,-0.305011,-0.103360,1.483887,0.178855
4,-0.083359,-0.022609,0.178529,1.80371,0.342193,1.80371,0.0,0.512726,0.0,0.000000,0.011951,-0.393514,0.149152,-0.088010,0.149152,-0.289337,0.272234,1.483887,0.178855


In [4]:
X_encoded[X_encoded < 0]=0
X_encoded.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.155052,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.0,0.194272,0.000000,0.011951,0.000000,0.0,0.000000,0.0,0.299633,0.000000,0.000000
1,0.000000,0.0,0.000000,0.00000,0.194272,0.00000,0.0,0.000000,0.0,0.417416,0.000000,0.417416,1.110563,0.0,1.110563,0.0,0.000000,0.000000,0.000000
2,0.011951,0.0,0.178529,0.00000,0.000000,0.00000,0.0,0.117311,0.0,0.194272,0.000901,0.194272,0.000000,0.0,0.000000,0.0,0.705098,0.000000,0.178855
3,0.000000,0.0,0.178529,0.00000,0.045852,0.00000,0.0,0.242475,0.0,0.000000,0.078200,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.483887,0.178855
4,0.000000,0.0,0.178529,1.80371,0.342193,1.80371,0.0,0.512726,0.0,0.000000,0.011951,0.000000,0.149152,0.0,0.149152,0.0,0.272234,1.483887,0.178855


In [5]:
X.shape

(5000, 19)

In [14]:
X_train=X_encoded
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
9,total_night_minutes,984.747141
6,total_eve_minutes,943.806814
8,total_eve_charge,902.033152
5,total_day_charge,860.955875
3,total_day_minutes,841.665647
15,number_customer_service_calls,833.029988
11,total_night_charge,578.709135
17,international_plan,451.047311
14,total_intl_charge,102.283438
12,total_intl_minutes,98.297089


In [15]:
y_train=y
x_train_chi = select_feature.transform(X_train)
x_train_chi.shape

(5000, 15)

In [8]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X_train,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3808 485 134 573
pod:  0.8104667609618105
pof:  0.11297460982995575
AUC:  0.8487460755659273


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 556 tasks      | elapsed:    1.0s


{'var_smoothing': 0.533669923120631}
GaussianNB(priors=None, var_smoothing=0.533669923120631)
0.9196


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    2.5s finished


In [32]:
{'var_smoothing': 1e-09}
GaussianNB(priors=None, var_smoothing=1e-09)
0.51704


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [16]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.533669923120631)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4156 137 268 439
pod:  0.620933521923621
pof:  0.031912415560214305
AUC:  0.7945105531817033
accuracy:  0.919


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

    

accuracy:  0.54812


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s


{'C': 1, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
0.9362


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    2.3s finished


In [18]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4202 91 228 479
pod:  0.6775106082036775
pof:  0.021197297926857674
AUC:  0.8281566551384099
accuracy:  0.9362


In [19]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression()
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

4205 88 228 479
pod:  0.6775106082036775
pof:  0.020498485907290937
AUC:  0.8285060611481933
accuracy:  0.9368


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.9264


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    3.0s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [21]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4232 61 311 396
pod:  0.5601131541725601
pof:  0.01420917773119031
AUC:  0.7729519882206849
accuracy:  0.9256


In [22]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4229 64 315 392
pod:  0.5544554455445545
pof:  0.014907989750757047
AUC:  0.7697737278968987
accuracy:  0.9242


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   26.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  2.8min finished


{'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9326


In [24]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4224 69 277 430
pod:  0.6082036775106082
pof:  0.01607267645003494
AUC:  0.7960655005302866
accuracy:  0.9308


In [25]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
#classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4207 86 306 401
pod:  0.5671852899575672
pof:  0.02003261122757978
AUC:  0.7735763393649937


In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  40 | elapsed:    0.0s remaining:    0.4s


{'criterion': 'gini', 'splitter': 'best'}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.8992


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished


In [31]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4064 229 278 429
pod:  0.6067892503536068
pof:  0.05334265082692756
AUC:  0.7767232997633396
accuracy:  0.8986


In [27]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4042 251 261 446
pod:  0.6308345120226309
pof:  0.05846727230375029
AUC:  0.7861836198594403


In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


{}
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
0.79167


In [32]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4201 92 240 467
pod:  0.6605374823196606
pof:  0.021430235266713255
AUC:  0.8195536235264737
accuracy:  0.9336


In [33]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4209 84 224 483
pod:  0.6831683168316832
pof:  0.019566736547868623
AUC:  0.8318007901419073
accuracy:  0.9384


In [34]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 149us/step - loss: 0.3476 - acc: 0.8808 - val_loss: 0.2203 - val_acc: 0.9122
Epoch 2/30
3599/3599 [==============================] - 0s 66us/step - loss: 0.2047 - acc: 0.9197 - val_loss: 0.2077 - val_acc: 0.9211
Epoch 3/30
3599/3599 [==============================] - 0s 59us/step - loss: 0.1871 - acc: 0.9286 - val_loss: 0.1937 - val_acc: 0.9256
Epoch 4/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.1796 - acc: 0.9316 - val_loss: 0.1862 - val_acc: 0.9256
Epoch 5/30
3599/3599 [==============================] - 0s 55us/step - loss: 0.1731 - acc: 0.9322 - val_loss: 0.1784 - val_acc: 0.9267
Epoch 6/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.1719 - acc: 0.9344 - val_loss: 0.1782 - val_acc: 0.9300
Epoch 7/30
3599/3599 [

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 160us/step - loss: 0.3571 - acc: 0.8733 - val_loss: 0.2267 - val_acc: 0.9144
Epoch 2/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.2118 - acc: 0.9169 - val_loss: 0.2019 - val_acc: 0.9267
Epoch 3/30
3599/3599 [==============================] - 0s 58us/step - loss: 0.1939 - acc: 0.9239 - val_loss: 0.1860 - val_acc: 0.9311
Epoch 4/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.1836 - acc: 0.9322 - val_loss: 0.1821 - val_acc: 0.9300
Epoch 5/30
3599/3599 [==============================] - 0s 56us/step - loss: 0.1798 - acc: 0.9336 - val_loss: 0.1769 - val_acc: 0.9311
Epoch 6/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.1728 - acc: 0.9336 - val_loss: 0.1799 - val_acc: 0.9300
Epoch 7/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.1661 - acc: 0.9405 - val_loss: 0.1775 - val_acc: 0.9311
Epoch 8

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 173us/step - loss: 0.3048 - acc: 0.8916 - val_loss: 0.1960 - val_acc: 0.9244
Epoch 2/30
3599/3599 [==============================] - 0s 52us/step - loss: 0.2019 - acc: 0.9230 - val_loss: 0.1986 - val_acc: 0.9200
Epoch 3/30
3599/3599 [==============================] - 0s 52us/step - loss: 0.1872 - acc: 0.9272 - val_loss: 0.1903 - val_acc: 0.9211
Epoch 4/30
3599/3599 [==============================] - 0s 57us/step - loss: 0.1859 - acc: 0.9233 - val_loss: 0.1922 - val_acc: 0.9222
Epoch 5/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.1785 - acc: 0.9275 - val_loss: 0.1846 - val_acc: 0.9278
Epoch 6/30
3599/3599 [==============================] - 0s 61us/step - loss: 0.1752 - acc: 0.9322 - val_loss: 0.1826 - val_acc: 0.9256
Epoch 7/30
3599/3599 [==============================] - 0s 64us/step - loss: 0.1703 - acc: 0.9314 - val_loss: 0.1844 - val_acc: 0.9300
Epoch 8

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 188us/step - loss: 0.3278 - acc: 0.8903 - val_loss: 0.2199 - val_acc: 0.9133
Epoch 2/30
3600/3600 [==============================] - 0s 55us/step - loss: 0.2089 - acc: 0.9208 - val_loss: 0.2039 - val_acc: 0.9211
Epoch 3/30
3600/3600 [==============================] - 0s 53us/step - loss: 0.2012 - acc: 0.9211 - val_loss: 0.1943 - val_acc: 0.9222
Epoch 4/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.1852 - acc: 0.9297 - val_loss: 0.1839 - val_acc: 0.9256
Epoch 5/30
3600/3600 [==============================] - 0s 54us/step - loss: 0.1834 - acc: 0.9294 - val_loss: 0.1813 - val_acc: 0.9289
Epoch 6/30
3600/3600 [==============================] - 0s 54us/step - loss: 0.1745 - acc: 0.9336 - val_loss: 0.1881 - val_acc: 0.9222
Epoch 7/30
3600/3600 [==============================] - 0s 53us/step - loss: 0.1754 - acc: 0.9306 - val_loss: 0.1762 - val_acc: 0.9256
Epoch 8

...

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 223us/step - loss: 0.3568 - acc: 0.8756 - val_loss: 0.2563 - val_acc: 0.9167
Epoch 2/30
3600/3600 [==============================] - 0s 57us/step - loss: 0.2366 - acc: 0.9108 - val_loss: 0.2069 - val_acc: 0.9244
Epoch 3/30
3600/3600 [==============================] - 0s 57us/step - loss: 0.2010 - acc: 0.9233 - val_loss: 0.1904 - val_acc: 0.9311
Epoch 4/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.1938 - acc: 0.9256 - val_loss: 0.1830 - val_acc: 0.9378
Epoch 5/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.1834 - acc: 0.9314 - val_loss: 0.1742 - val_acc: 0.9367
Epoch 6/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.1748 - acc: 0.9367 - val_loss: 0.1731 - val_acc: 0.9333
Epoch 7/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.1706 - acc: 0.9372 - val_loss: 0.1666 - val_acc: 0.9433
Epoch 8

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 241us/step - loss: 0.3499 - acc: 0.8733 - val_loss: 0.2434 - val_acc: 0.9211
Epoch 2/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.2323 - acc: 0.9144 - val_loss: 0.2141 - val_acc: 0.9244
Epoch 3/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.2108 - acc: 0.9197 - val_loss: 0.1990 - val_acc: 0.9278
Epoch 4/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.1981 - acc: 0.9231 - val_loss: 0.1954 - val_acc: 0.9211
Epoch 5/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.1888 - acc: 0.9253 - val_loss: 0.1865 - val_acc: 0.9189
Epoch 6/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1834 - acc: 0.9294 - val_loss: 0.1794 - val_acc: 0.9322
Epoch 7/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.1784 - acc: 0.9333 - val_loss: 0.1942 - val_acc: 0.9256
Epoch 8

 [0.]]
pod 1st:  0.8450704225352113
Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 256us/step - loss: 0.3348 - acc: 0.8800 - val_loss: 0.2749 - val_acc: 0.8779
Epoch 2/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.2208 - acc: 0.9131 - val_loss: 0.2024 - val_acc: 0.9267
Epoch 3/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.2016 - acc: 0.9208 - val_loss: 0.1906 - val_acc: 0.9256
Epoch 4/30
3600/3600 [==============================] - 0s 64us/step - loss: 0.1909 - acc: 0.9217 - val_loss: 0.1905 - val_acc: 0.9234
Epoch 5/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1827 - acc: 0.9292 - val_loss: 0.1764 - val_acc: 0.9323
Epoch 6/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1756 - acc: 0.9328 - val_loss: 0.1805 - val_acc: 0.9290
Epoch 7/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1790 - acc: 0.9269 - val_lo

pod 1st:  0.6571428571428571
Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 278us/step - loss: 0.3233 - acc: 0.8897 - val_loss: 0.2632 - val_acc: 0.9001
Epoch 2/30
3600/3600 [==============================] - 0s 60us/step - loss: 0.2194 - acc: 0.9169 - val_loss: 0.2003 - val_acc: 0.9223
Epoch 3/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.1977 - acc: 0.9219 - val_loss: 0.1942 - val_acc: 0.9245
Epoch 4/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1832 - acc: 0.9275 - val_loss: 0.1934 - val_acc: 0.9312
Epoch 5/30
3600/3600 [==============================] - 0s 65us/step - loss: 0.1752 - acc: 0.9322 - val_loss: 0.1829 - val_acc: 0.9312
Epoch 6/30
3600/3600 [==============================] - 0s 60us/step - loss: 0.1701 - acc: 0.9336 - val_loss: 0.1908 - val_acc: 0.9301
Epoch 7/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.1686 - acc: 0.9369 - val_loss: 0.1

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 300us/step - loss: 0.3364 - acc: 0.8797 - val_loss: 0.2251 - val_acc: 0.9145
Epoch 2/30
3600/3600 [==============================] - 0s 71us/step - loss: 0.2231 - acc: 0.9117 - val_loss: 0.2012 - val_acc: 0.9245
Epoch 3/30
3600/3600 [==============================] - 0s 61us/step - loss: 0.1991 - acc: 0.9217 - val_loss: 0.1985 - val_acc: 0.9312
Epoch 4/30
3600/3600 [==============================] - 0s 60us/step - loss: 0.1853 - acc: 0.9297 - val_loss: 0.1892 - val_acc: 0.9312
Epoch 5/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.1803 - acc: 0.9283 - val_loss: 0.1803 - val_acc: 0.9312
Epoch 6/30
3600/3600 [==============================] - 0s 61us/step - loss: 0.1763 - acc: 0.9314 - val_loss: 0.1857 - val_acc: 0.9323
Epoch 7/30
3600/3600 [==============================] - 0s 68us/step - loss: 0.1671 - acc: 0.9361 - val_loss: 0.1908 - val_acc: 0.9390
Epoch 8

In [35]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 3599 samples, validate on 900 samples
Epoch 1/5
3599/3599 [==============================] - 2s 454us/step - loss: 0.4436 - acc: 0.8586 - val_loss: 0.3353 - val_acc: 0.8589
Epoch 2/5
3599/3599 [==============================] - 0s 86us/step - loss: 0.3038 - acc: 0.8616 - val_loss: 0.2801 - val_acc: 0.8678
Epoch 3/5
3599/3599 [==============================] - 0s 86us/step - loss: 0.2752 - acc: 0.8808 - val_loss: 0.3091 - val_acc: 0.8833
Epoch 4/5
3599/3599 [==============================] - 0s 90us/step - loss: 0.2712 - acc: 0.8922 - val_loss: 0.2687 - val_acc: 0.8978
Epoch 5/5
3599/3599 [==============================] - 0s 86us/step - loss: 0.2634 - acc: 0.8969 - val_loss: 0.2761 - val_acc: 0.8989
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

3599/3599 [==============================] - 0s 80us/step - loss: 0.2614 - acc: 0.9022 - val_loss: 0.2680 - val_acc: 0.9078
y22_pred:  [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 2s 610us/step - loss: 0.4554 - acc: 0.8586 - val_loss: 0.3525 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 0s 84us/step - loss: 0.3146 - acc: 0.8600 - val_loss: 0.2829 - val_acc: 0.8633
Epoch 3/5
3600/3600 [==============================] - 0s 84us/step - loss: 0.2788 - acc: 0.8733 - val_loss: 0.2652 - val_acc: 0.8856
Epoch 4/5
3600/3600 [==============================] - 0s 90us/step - loss: 0.2672 - acc: 0.8953 - val_loss: 0.2681 - val_acc: 0.9056
Epoch 5/5
3600/3600 [==============================] - 0s 87us/step - loss: 0.2648 - acc: 0.8983 - val_loss: 0.2728 - val_acc: 0.9022
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

3600/3600 [==============================] - 0s 92us/step - loss: 0.2615 - acc: 0.8964 - val_loss: 0.2630 - val_acc: 0.9012
y22_pred:  [[1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 